## 把抓到的TF-IDF 關鍵詞存入資料庫

In [1]:
#把使用的套件都匯入
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['test']

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.492 seconds.
Prefix dict has been built succesfully.


## Get Data

In [2]:
import time
tic = time.clock()

title=[]
content =[]
#拿資料
for post in collection.find(): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    title.append(post['title'])
#總文章數量
print "文章數:",len(title)
toc = time.clock()
print "執行時間:",(toc - tic)/60

文章數: 7135
執行時間: 0.247664899644


## Get Weight & features

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray()

#特徵值總共有多少個
features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  
print "特徵值數量:",len(features)

Feature_num 123652


## Get Top TF-IDF

In [58]:
import time
tic = time.clock()

top_features = []
for n in range(0,7134):  #迴圈參考上面的總文章數
    indices = np.argsort(weight[n])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
    # 看TOP多少的詞
    top_n = 30
    top_features.append([features[i] for i in indices[:top_n]])    #這邊的寫法會讓關鍵字中間會有空白
    
toc = time.clock()
print "執行時間:",(toc - tic)/60

print len(top_features) #可加一行讓每組關鍵字中間是 / 
print type(top_features)

執行時間: 0.209206105439
7134
<type 'list'>


## Put into Database

In [59]:
import time
tic = time.clock()

obj = collection.find()  #這行一定要加
for i in range(0,7134):  #迴圈參考上面的總文章數
    collection.update({"title":obj[i]["title"]},{"$set":{"tfidf1":top_features[i]}}) #collection.update即可

toc = time.clock()
print "執行時間:",(toc - tic)/60

執行時間: 0.332543149083


In [57]:
# Check
#存放方式 "tfidf1" : [ "最新民調", "成則", "強國", "大陸人", "釣魚台主權", "中日關係", "爭端", "中日", "環球時報"]

obj = collection.find()
print obj[110]['tfidf'][2]  #看第幾篇新聞的tf-idf屬性以及看第幾個元素
print top_features[1][2]

多則
五月天
